In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import mxnet as mx
import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
No boto3
No vtk
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_anchor.txt


Setting environment for Precision WorkStation


File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed/MD653/MD653_sorted_filenames.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD653/MD653_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_anchor.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_cropbox.txt
File does not exist: /media/yuncong/BstemAtlasData/CSHL_data_processed//MD652/MD652_cropbox.txt


In [2]:
MODEL_ROOTDIR = '/home/yuncong/mxnet_models/'

In [3]:
# Does not work for the mxnet version on 3/30/2017
# model_dir_name = 'Sat16ClassFinetuned'
# model_name = 'Sat16ClassFinetuned'
# model_iteration = 10
# output_symbol_name = 'flatten_output'
# mean_img = np.load(os.path.join(MODEL_ROOTDIR, model_dir_name, 'saturation_mean_224.npy'))

# model_dir_name = 'vgg16'
# model_name = 'vgg16'
# model_iteration = 0
# output_symbol_name = 'fc8_output'

# model_dir_name = 'vgg19'
# model_name = 'vgg19'
# model_iteration = 0
# output_symbol_name = 'fc8_output'

model_dir_name = 'inception-bn'
model_name = 'Inception-BN'
model_iteration = 126
output_symbol_name = 'flatten_output'
mean_img = mx.nd.load(os.path.join(MODEL_ROOTDIR, model_dir_name, 'mean_224.nd'))['mean_img'].asnumpy()

model0 = mx.model.FeedForward.load(os.path.join(MODEL_ROOTDIR, model_dir_name, model_name), model_iteration, ctx=mx.gpu())
flatten_output = model0.symbol.get_internals()[output_symbol_name]
model = mx.model.FeedForward(ctx=mx.gpu(), 
                           symbol=flatten_output, 
                           arg_params = model0.arg_params, 
                           aux_params = model0.aux_params,
                           allow_extra_params = True)

# model_dir_name = 'inception-bn-sat'
# model_name = 'inception-bn-sat'
# model_iteration = 0
# mean_img = np.load(os.path.join(MODEL_ROOTDIR, model_dir_name, 'mean_224.npy'))

# model = mx.model.FeedForward.load(os.path.join(MODEL_ROOTDIR, model_dir_name, model_name), model_iteration, ctx=mx.gpu())

/home/yuncong/mxnet/python/mxnet/model.py:870: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
  **kwargs)
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.


In [4]:
# Visualize network topology

# a = mx.viz.plot_network(flatten_output, shape={"data":(1, 1, 224, 224)}, node_attrs={"shape":'rect',"fixedsize":'false'})
# a.render(model_name)

In [5]:
patch_size = 224
half_size = patch_size/2
stride = 56

In [ ]:
def convert_to_saturation(img, rescale=True):
    """
    Generate saturation channel as a grayscale image.
    """
    ma = img.max(axis=-1)
    mi = img.min(axis=-1)
#     sys.stderr.write('compute min and max color components: %.2f seconds\n' % (time.time() - t1)) # ~5s

#     t1 = time.time()
    s = np.nan_to_num(mi/ma.astype(np.float))
#     sys.stderr.write('min oiver max: %.2f seconds\n' % (time.time() - t1)) # ~2s

#     t1 = time.time()
    if rescale:
        pmax = s.max()
        pmin = s.min()
        s = (s - pmin) / (pmax - pmin)
#     sys.stderr.write('rescale: %.2f seconds\n' % (time.time() - t1)) # ~3s

    return img_as_ubyte(s)

In [21]:
for stack in all_stacks:
    
    if stack in ['MD585', 'MD589', 'MD594', 'MD657']:
        continue
# for stack in ['MD585']:

    print stack

    section_to_filename = metadata_cache['sections_to_filenames'][stack]

    image_width, image_height = metadata_cache['image_shape'][stack]
    grid_spec = (patch_size, stride, image_width, image_height)

    sample_locations = grid_parameters_to_sample_locations(grid_spec=grid_spec)

    first_detect_sec, last_detect_sec = metadata_cache['section_limits'][stack]

    bar = show_progress_bar(first_detect_sec, last_detect_sec)
# 
#     for sec in range(201, 202):
    for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in range(143, last_detect_sec+1):
        
        if is_invalid(stack=stack, sec=sec):
            continue
            
        if section_to_filename[sec].split('-')[1][0] == 'F':
            continue
                
        bar.value = sec
        
        sys.stderr.write('\nSection %d.\n' % sec)

        # Use grids only on mask.
#         t = time.time()
#         mask_tb = DataManager.load_thumbnail_mask_v2(stack=stack, section=sec)
#         indices_roi = locate_patches_v2(grid_spec=grid_spec, mask_tb=mask_tb)
#         sys.stderr.write('locate patches: %.2f seconds\n' % (time.time() - t))       
        
        # Use grids on the entire frame.
        indices_roi = range(len(sample_locations))

        n = len(indices_roi)
        sys.stderr.write('%d samples.\n' % n)

        sample_locations_roi = sample_locations[indices_roi]
        
        t = time.time()
        im = img_as_ubyte(imread(DataManager.get_image_filepath(stack=stack, section=sec, version='cropped', resol='lossless')))
        sys.stderr.write('load image: %.2f seconds\n' % (time.time() - t)) # ~ 35s
        
#         t = time.time()
#         sat = convert_to_saturation(im)
#         del im
#         sys.stderr.write('Convert to saturation: %.2f seconds\n' % (time.time() - t)) # ~ 35s
#         sat = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='saturation'))
        

        t = time.time()

#         patches = np.array([sat[y-half_size:y+half_size, x-half_size:x+half_size]
#                             for x, y in sample_locations_roi]) # n x 224 x 224
#         patches_mean_subtracted = patches - mean_img
#         patches_mean_subtracted_input = patches_mean_subtracted[:, None, :, :] # n x 1 x 224 x 224

        patches = np.array([im[y-half_size:y+half_size, x-half_size:x+half_size]
                            for x, y in sample_locations_roi]) # n x 224 x 224 x 3
        patches_mean_subtracted_input = np.rollaxis(patches, 3, 1) - mean_img # n x 3 x 224 x 224

        sys.stderr.write('extract patches: %.2f seconds\n' % (time.time() - t)) # ~ 20s / 140k patches

        # Increase batch_size to 500 does not save any time.
        batch_size = 256 
#         batch_size = 16

        # For mxnet 0.9.4, using 80000 patches in a NDArrayIter works but using 90000 patches causes error:
        # TypeError: Invalid type '<type 'numpy.ndarray'>' for data, should be NDArray or numpy.ndarray

        t1 = time.time()
        features = np.empty((n, 1024))
        
        n_each_iter = 20000
        for b in range(0, n, n_each_iter):
            data_iter = mx.io.NDArrayIter(
                patches_mean_subtracted_input[b:b+n_each_iter], 
                np.zeros((n_each_iter, ), np.int), # labels are not important since it is just feed-forward
                batch_size = batch_size,
                shuffle=False)

#             if b == 0:
#                 features = model.predict(data_iter)
#             else:
#                 features = np.vstack([features, model.predict(data_iter)])
            features[b:b+n_each_iter] = model.predict(data_iter)
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t1)) # 220s-250s
        
#         data_iter = mx.io.NDArrayIter(
#             patches_mean_subtracted_input, 
#             np.zeros((n, ), np.int), # labels are not important since it is just feed-forward
#             batch_size = batch_size,
#             shuffle=False
#         )

#         t = time.time()
#         features = model.predict(data_iter)
#         sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))

    # The first time when CUDA needs to load model is very slow ~350s, but later runs are faster ~90s
        
    # Had to modify [model]-symbol.json according to this https://github.com/dmlc/mxnet/issues/2718
    
    # Out of memory after about 20 sections - had to modify storage code according to https://github.com/dmlc/mxnet/issues/3055
    #    - don't think increasing kPoolThreshold to over 4GB will be beneficial, 
    # since the computation time is similar to before limiting the pool, computation is most likely compute-bound.
    # Issue solved by developers.
        
        t = time.time()

        features_fp = DataManager.get_dnn_features_filepath(stack=stack, model_name=model_name, 
                                                          section=sec)
        create_parent_dir_if_not_exists(features_fp)
        save_hdf(features.astype(np.float16), features_fp) # bloscpack produces files of similar size

        feature_locs_fp = DataManager.get_dnn_feature_locations_filepath(stack=stack, model_name=model_name, 
                                                  section=sec)
        np.savetxt(feature_locs_fp, np.c_[indices_roi, sample_locations_roi], fmt='%d %d %d')

        sys.stderr.write('save: %.2f seconds\n' % (time.time() - t)) # ~.5s
        
        del patches_mean_subtracted_input, patches, im
#         del patches_mean_subtracted_input, patches_mean_subtracted, patches, sat


Section 82.
71300 samples.
load image: 53.12 seconds
extract patches: 42.74 seconds
predict: 180.05 seconds
save: 5.06 seconds

Section 83.
71300 samples.
load image: 45.63 seconds
extract patches: 73.70 seconds
predict: 166.45 seconds
save: 4.11 seconds

Section 84.
71300 samples.
load image: 47.38 seconds
extract patches: 31.81 seconds
predict: 154.93 seconds
save: 3.74 seconds

Section 85.
71300 samples.
load image: 49.74 seconds
extract patches: 31.65 seconds
predict: 161.17 seconds
save: 4.19 seconds

Section 86.
71300 samples.
load image: 51.58 seconds
extract patches: 30.94 seconds
predict: 176.65 seconds
save: 3.93 seconds

Section 87.
71300 samples.
load image: 50.05 seconds
extract patches: 37.51 seconds
predict: 175.88 seconds
save: 4.26 seconds

Section 88.
71300 samples.
load image: 49.99 seconds
extract patches: 47.12 seconds
predict: 186.94 seconds
save: 4.14 seconds

Section 89.
71300 samples.
load image: 48.82 seconds
extract patches: 30.09 seconds
predict: 153.55 sec

MD590
MD591

save: 4.01 seconds

Section 98.
65142 samples.
load image: 57.09 seconds
extract patches: 27.22 seconds
predict: 141.21 seconds
save: 4.27 seconds

Section 99.
65142 samples.
load image: 57.52 seconds
extract patches: 27.94 seconds
predict: 130.80 seconds
save: 4.18 seconds

Section 100.
65142 samples.
load image: 59.34 seconds
extract patches: 26.91 seconds
predict: 155.21 seconds
save: 4.23 seconds

Section 101.
65142 samples.
load image: 55.80 seconds
extract patches: 26.99 seconds
predict: 143.73 seconds
save: 4.03 seconds

Section 102.
65142 samples.
load image: 56.57 seconds
extract patches: 27.40 seconds
predict: 135.63 seconds
save: 3.80 seconds

Section 103.
65142 samples.
load image: 56.38 seconds
extract patches: 27.31 seconds
predict: 159.86 seconds
save: 4.19 seconds

Section 104.
65142 samples.
load image: 59.82 seconds
extract patches: 27.73 seconds
predict: 149.62 seconds
save: 4.35 seconds

Section 105.
65142 samples.
load image: 55.48 seconds
extract patches: 27.29 se


MD592

save: 4.17 seconds

Section 91.
67144 samples.
load image: 52.43 seconds
extract patches: 27.82 seconds
predict: 156.59 seconds
save: 4.02 seconds

Section 92.
67144 samples.
load image: 56.76 seconds
extract patches: 27.61 seconds
predict: 160.81 seconds
save: 4.11 seconds

Section 93.
67144 samples.
load image: 52.66 seconds
extract patches: 27.85 seconds
predict: 147.16 seconds
save: 3.99 seconds

Section 94.
67144 samples.
load image: 56.88 seconds
extract patches: 27.85 seconds
predict: 157.73 seconds
save: 4.40 seconds

Section 95.
67144 samples.
load image: 43.76 seconds
extract patches: 30.03 seconds
predict: 155.19 seconds
save: 3.44 seconds

Section 96.
67144 samples.
load image: 50.31 seconds
extract patches: 27.72 seconds
predict: 150.63 seconds
save: 3.86 seconds

Section 97.
67144 samples.
load image: 48.42 seconds
extract patches: 27.91 seconds
predict: 144.77 seconds
save: 3.90 seconds

Section 98.
67144 samples.
load image: 51.34 seconds
extract patches: 29.98 seconds


...


MD603

save: 4.40 seconds

Section 61.
87690 samples.
load image: 55.26 seconds
extract patches: 47.29 seconds
predict: 211.94 seconds
save: 4.14 seconds

Section 62.
87690 samples.
load image: 54.04 seconds
extract patches: 39.17 seconds
predict: 210.75 seconds
save: 4.31 seconds

Section 63.
87690 samples.
load image: 53.73 seconds
extract patches: 61.92 seconds
predict: 214.00 seconds
save: 4.48 seconds

Section 64.
87690 samples.
load image: 54.48 seconds
extract patches: 39.14 seconds
predict: 204.79 seconds
save: 4.39 seconds

Section 65.
87690 samples.
load image: 63.26 seconds
extract patches: 59.98 seconds
predict: 221.03 seconds
save: 4.58 seconds

Section 66.
87690 samples.
load image: 56.29 seconds
extract patches: 39.04 seconds
predict: 205.02 seconds
save: 3.95 seconds

Section 67.
87690 samples.
load image: 53.20 seconds
extract patches: 42.01 seconds
predict: 202.36 seconds
save: 4.39 seconds

Section 68.
87690 samples.
load image: 52.18 seconds
extract patches: 48.82 seconds



MD635
MD653


save: 4.59 seconds


KeyError: 'MD653'

In [17]:
from distributed_utilities import *

In [20]:
transfer_data_synced(os.path.join('CSHL_patch_features', model_name, 'MD589'), 
                    from_hostname='workstation',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive /media/yuncong/BstemAtlasData/CSHL_patch_features/Inception-BN/MD589 s3://mousebrainatlas-data/CSHL_patch_features/Inception-BN/MD589


Child returned 0


# Sync with Gordon

In [ ]:
cmd = 'cd /media/yuncong/BstemAtlasData/CSHL_patch_features/Sat16ClassFinetuned/%(stack)s/ && \
rsync -r . yuncong@oasis-dm.sdsc.edu:/home/yuncong/csd395/CSHL_patch_features/Sat16ClassFinetuned/%(stack)s' % \
{'stack': stack}

os.system(cmd)